# Automating My Daily Lineups: Start Active Players
Automatically setting my NBA Fantasy lineups by Jaume Clave

December 19, 2020

I've been playing Fantasy NBA for 5 years now. Before the real NBA season starts we get a group
of friends to join our 12 team, 9cat h2h NBA Fantasy league on Yahoo. It's good fun to build out
a team and see how players perform throughout the year.

It can get quite competitive between the team owners and leagues in the past have been won on
small margins. Each owner tries to get as many advantages as they can. People set up Twitter
alerts so that they are first to know about an injury or trade. Team gamble on *sleepers* in hope
 that a waiver wire add can explode into a huge signing.

One of the most commons ways of losing a h2h match up is forgetting to set your lineups.
Unfortunately, if you leave one of your players on the bench, you miss out on all the points that
 player makes for you. This often happens as this is something that needs to be done and checked
 daily. Life gets in the way and one forgets.

This project aims to fix this. This project creates a script using Selenium that logs into my
Yahoo Fantasy and automatically sets my players as *active*. Meaning their in-game performances
will count towards my totals. This script will be executed daily using Windows Task Scheduler.

## Index

[Selenium](#Selenium) <br>
i. [ChromeDriver](#ChromeDriver) <br>
ii. [Finding Elements With Selenium](Finding-Elements-With-Selenium)

[Python Scripting](#Python-Scripting) <br>
i. [Windows Task Scheduler](#Windows-Task-Scheduler) <br>
ii. [if \_\_name\_\_ == "\_\_main\_\_":](#if-\_\_name\_\_-==-"\_\_main\_\_":)

[Conclusion](#Conclusion)

[Further Reading](#Further-Reading)


## Selenium
Selenium WebDriver is one of the most popular tools for Web UI Automation. The selenium package
is  used to automate web browser interaction from Python. Web UI Automation refers to the
automatic  execution of the actions preformed in a web browser window like navigating to a
website, filing forms that deal with input to text boxes, with radio buttons, downloading of
images  and files, submission of forms and general website interactions. With Selenium a user is
 able to automate all this stuff; automate the boring stuff!

This section will use Selenium to automate the three steps involved in setting a daily lineups:

    1. Authenticate on Yahoo and navigate to the "My Team" page
    2. Find the "Start Active Players" button and click it
    3. Find the "Start" button in the popup and click it

Each step will be broken down and visualised.

In [1]:
# Import modules
from selenium import webdriver
from selenium.webdriver import Chrome
import time
import warnings
warnings.filterwarnings('ignore')

### ChromeDriver
In order to automate with selenium on Google Chrome, chromedriver.exe must be downloaded because
a  path to this needs to be executable on your computer. WebDriver is an open source tool for
automated testing of webapps across many browsers. It provides capabilities for navigating to web
  pages, user input, JavaScript execution, and more. ChromeDriver is a standalone server that
  implements the W3C WebDriver standard.

The first step is to load the Yahoo My Team page using the chromedriver. Yahoo requires you to
authenticate and login when using the service on a new browser/device for security purposes.
This login was completed manually once on the ChromeDriver. From then on, because a user profile
is loaded using the 'user-data-dir=""' argument this will not have to be repeated. The page can
now be accessed using the .get() method which loads the URL. The time.sleep() function is used to
 add delay in the execution of the program. It used to halt the execution of the program for
 given time in seconds and therefore give the web page enough time to load before moving on to
 the next line of code.


In [2]:
# Chrome arguments
CHROME_USER_DATA_DIRECTORY_ARGUMENT = r"user-data-dir=C:\Users\Jaume\AppData\Local\Google\Chrome" \
                                      r"\User Data\Python"
CHROME_DISABLE_NOTIFICATION_ARGUMENT = "--disable-notifications"
CHROME_EXECUTABLE_PATH =  r"C:\Users\Jaume\Documents\Python " \
                          r"Projects\yahoo_fantasy_lineups\chromedriver.exe"

SLEEP_SECONDS = 1

In [3]:
# Load Profile
options = webdriver.ChromeOptions()

# Load Chrome profile and download directory
options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

# Chrome with profile
driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)
time.sleep(SLEEP_SECONDS)

In [4]:
# Load "My Team" page
MY_TEAM_URL = "https://basketball.fantasysports.yahoo.com/nba/55374/6"

# Get URL
driver.get(MY_TEAM_URL)
time.sleep(SLEEP_SECONDS)

<img src="http://drive.google.com/uc?export=view&id=1NNjQc_r7MsA4gj7Nca3atwOI_rNiCXoJ" />

### Finding Elements With Selenium
There are many ways to find an element on a page using selenium, all require some basic
understanding of HTML. XPath is the language used for locating nodes in an XML document. As HTML
 can be an implementation of XML (XHTML), we can leverage this powerful language to target
 elements  in the WhatsApp Web application. XPath extends beyond (as well as supporting) the
 simple  methods of locating by id or name attributes, and opens up all sorts of new
 possibilities  such as locating the third checkbox on the page.

One of the main reasons for using XPath is when you don’t have a suitable id or name attribute
for  the element you wish to locate. You can use XPath to either locate the element in absolute
terms (not advised), or relative to an element that does have an id or name attribute. XPath
locators can also be used to specify elements via attributes other than id and name.

Absolute XPaths contain the location of all elements from the root (HTML) and as a result are
likely to fail with only the slightest adjustment to the application. By finding a nearby element
  with an id or name attribute (ideally a parent element) you can locate your target element
  based  on the relationship. This is much less likely to change and can make your tests more
  robust.

The first element that needs to be found is the button that contains the "Start Active Players"
text. We can find the XPATH by inspecting the element. This button, when clicked, loads a popup
which we will need to perform another action on. Once the initial button is clicked, we delay the
execution of a system slightly to allow for new elements to load and populate the HTML.

In [5]:
# Click "Start Active Players" button
START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT = "//*[(text()='Start Active Players')]"

# Find and click
start_active_players_button = driver.find_element_by_xpath(START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
start_active_players_button.click()
time.sleep(SLEEP_SECONDS)

<img src="http://drive.google.com/uc?export=view&id=166BW20jWjch4RVLEI_5gCFDbz64_Vc8a" />

In [6]:
# Click "Rest of Week Start" button
REST_OF_WEEK_START_BUTTON_XPATH_TEXT = "//*[(text()='Start')]"

# Find and click
start_rest_of_week_button = driver.find_element_by_xpath(REST_OF_WEEK_START_BUTTON_XPATH_TEXT)
start_rest_of_week_button.click()

driver.close()

<img src="http://drive.google.com/uc?export=view&id=1gaAK0Ea9OmGc5wIuiWll7FvHk3dO0leT" />

Following the successful player activation message the driver and window are closed. The Python
logic works and this process must now be optimised so that it can automatically be executed daily
 using Windows Task Scheduler.

## Python Scripting
The word script is used to refer to a file containing a logical sequence of orders, or a batch
processing file. This is usually a simple program. Scripts are always processed by some kind of
interpreter, which is responsible for executing each command sequentially.

A plain text file containing Python code that is intended to be directly executed by the user is
usually called script, which is an informal term that means top-level program file. The
collection of commands s in a file designed to be executed like a program. The file can of course
 contain functions and import various modules, but the idea is that it will be run or executed
 from the command line or from within a Python interactive shell to perform a specific task.
 Often a script first contains a set of function definitions and then has the main program that
 might call the functions.

The script is used to automate actions and is what is needed to automate this process. The script
 is the Python file that will be executed by Task Scheduler daily. The code used above to login,
 and click buttons will transform into functions which will be contained by a *main* function
  called ```set_daily_yahoo_lineups()```. This function will be responsible for executing the
  entire process.

In [7]:
# Functions
def open_yahoo_my_team():
    """
    Function opens the Yahoo "My Team" web page. ChromeDriver options are loaded which facilitate
    the Yahoo login. The driver is returned.
    :return driver: ChromeDriver used to open Yahoo session
    """
    # Load Profile
    options = webdriver.ChromeOptions()

    # Load Chrome profile and download directory
    options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
    options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

    # Chrome with profile
    driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)

    driver.get(MY_TEAM_URL)
    time.sleep(SLEEP_SECONDS)

    return driver

def click_button_yahoo_my_team(driver, button_xpath_text):
    """
    Function finds a webelement by XPATH and automatically clicks it.
    :param driver: ChromeDriver used to open Yahoo session
    :param button_xpath_text: XPATH of desired element
    :return driver: ChromeDriver used to open Yahoo session
    """
    webelement_button = driver.find_element_by_xpath(button_xpath_text)
    webelement_button.click()
    time.sleep(SLEEP_SECONDS)

    return driver

The function below will contain the functions above and will be the called on everyday to
automate the daily lineups process.

In [8]:
# Combining the functions
def set_daily_yahoo_lineups():
    """
    Function ties the entire process. It first logins to Yahoo via a ChromeDriver. It then finds
    the buttons required to be clicked to activate starting players.
    :return: None
    """
    driver = open_yahoo_my_team()
    click_button_yahoo_my_team(driver, START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
    click_button_yahoo_my_team(driver, REST_OF_WEEK_START_BUTTON_XPATH_TEXT)
    driver.close()


    return None

# Testing the function
set_daily_yahoo_lineups()

### Windows Task Scheduler
Task Scheduler is a component of Microsoft Windows that provides the ability to schedule the
launch  of programs or scripts at pre-defined times or after specified time intervals: job
scheduling. The tools used to create and run virtually any task automatically. Typically, the
system and certain apps use the scheduler to automate maintenance tasks (such as disk
defragmentation, disk cleanup, and updates), but anyone user can run it. With this experience,
can  start applications, run commands, and execute scripts at a particular day and time, or
you can also trigger tasks when a specific event occurs.

Task Scheduler works by keeping tabs of the time and events on your computer and executes the
task as soon as the condition is met. This script will be executed at 11am each day.

A GIF of how the process works using the Windows Task Scheduler, and an email being received
 is shown below:
 

<img src="http://drive.google.com/uc?export=view&id=1zZaVEmKVBcXWrJigOq2qcRWXHdS3xWge" />

### if \_\_name\_\_ == "\_\_main\_\_":
When a Python interpreter reads a Python file. it first sets a few special variables. Then it
executes the code from the file. One of those variables is called ```__name__```.

When the interpreter runs a module, the ```__name__``` variable will be set as ```__main__``` if
the module that is being run is the main program.

Advantages :

- Every Python module has it’s __name__ defined and if this is ‘__main__’, it implies that the
module is being run standalone by the user, and we can do corresponding appropriate actions.
- If you import this script as a module in another script, the __name__ is set to the name of the
script/module.
- Python files can act as either reusable modules, or as standalone programs.
- if __name__ == “main”: is used to execute some code only if the file was run directly, and not
imported.

By adding if ```\_\_name\_\_ == "\_\_main\_\_":``` to the end of the Python script, when the
script is executed the code following it will be run.

In [9]:
# Putting it all together
CHROME_USER_DATA_DIRECTORY_ARGUMENT = r"user-data-dir=C:\Users\Jaume\AppData\Local\Google\Chrome" \
                                      r"\User Data\Python"
CHROME_DISABLE_NOTIFICATION_ARGUMENT = "--disable-notifications"
CHROME_EXECUTABLE_PATH =  r"C:\Users\Jaume\Documents\Python " \
                          r"Projects\yahoo_fantasy_lineups\chromedriver.exe"

MY_TEAM_URL = "https://basketball.fantasysports.yahoo.com/nba/55374/6"

SLEEP_SECONDS = 1

START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT = "//*[(text()='Start Active Players')]"
REST_OF_WEEK_START_BUTTON_XPATH_TEXT = "//*[(text()='Start')]"


def open_yahoo_my_team():
    """
    Function opens the Yahoo "My Team" web page. ChromeDriver options are loaded which facilitate
    the Yahoo login. The driver is returned.
    :return driver: ChromeDriver used to open Yahoo session
    """
    # Load Profile
    options = webdriver.ChromeOptions()

    # Load Chrome profile and download directory
    options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
    options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

    # Chrome with profile
    driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)

    driver.get(MY_TEAM_URL)
    time.sleep(SLEEP_SECONDS)

    return driver


def click_button_yahoo_my_team(driver, button_xpath_text):
    """
    Function finds a webelement by XPATH and automatically clicks it.
    :param driver: ChromeDriver used to open Yahoo session
    :param button_xpath_text: XPATH of desired element
    :return driver: ChromeDriver used to open Yahoo session
    """
    webelement_button = driver.find_element_by_xpath(button_xpath_text)
    webelement_button.click()
    time.sleep(SLEEP_SECONDS)

    return driver


def set_daily_yahoo_lineups():
    """
    Function ties the entire process. It first logins to Yahoo via a ChromeDriver. It then finds
    the buttons required to be clicked to activate starting players.
    :return: None
    """
    driver = open_yahoo_my_team()
    click_button_yahoo_my_team(driver, START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
    click_button_yahoo_my_team(driver, REST_OF_WEEK_START_BUTTON_XPATH_TEXT)
    driver.close()

    return None


if __name__ == "__main__":
    set_daily_yahoo_lineups()


## Conclusion
Automating tasks is made simple with Python and its abundance of modules. Selenium helps automate
 browser interactions, and it was used throughout this project to navigate the Yahoo NBA Fantasy
 website.

The Python script will be run everyday so that I don't have to manually set my players. This
saves me time, and I can sleep easy knowing I won't lose valuable points from not having set my
lineups. The next steps are to make the script more intelligent. Two ways come to mind. A safety
measure needs to be introduced that should alert me if the script does not run or fails in its
execution. A push notification via WhatsApp, again automated by Selenium, or an email alerting me
 of the issue would prove useful. Another way the script could improve would be by ensuring that
 it set the best available players to play each day. There are only 10 active roster spots and
 some times 11 or more players play on any given day. An optimisation task might be introduced to
  ensure that the best player for the best position is introduced into the active line up!

## Further Reading
#### Selenium
[Selenium Documentaion](#https://selenium-python.readthedocs.io/) <br>
[Selenium Turotial](#https://www.guru99.com/selenium-python.html) <br>
[Web Scraping Using Selenium](#https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab)

#### Windows Task Scheduler¶
[Wikipedia - Windows Task Scheduler](#https://en.wikipedia.org/wiki/Windows_Task_Scheduler) <br>
[Creating Automated Tasks](#https://www.windowscentral.com/how-create-automated-task-using-task-scheduler-windows-10) <br>
[Step by Step Tasks](#https://www.digitalcitizen.life/how-create-task-basic-task-wizard)



